In [1]:
from __future__ import print_function
import cv2 as cv
import argparse

## [create]
#create Background Subtractor objects
#backSub = cv.createBackgroundSubtractorMOG2()

kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
backSub = cv.bgsegm.createBackgroundSubtractorGMG()

## [capture]
capture = cv.VideoCapture('videos/IMG_2164.MOV')
if not capture.isOpened():
    print('Unable to open: ' + args.input)
    exit(0)
## [capture]

while True:
    ret, frame = capture.read()
    if frame is None:
        break

    ## [apply]
    #update the background model
    fgMask = backSub.apply(frame)
    fgMask = cv.morphologyEx(fgMask, cv.MORPH_OPEN, kernel)
    ## [apply]

    ## [display_frame_number]
    #get the frame number and write it on the current frame
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    ## [display_frame_number]

    ## [show]
    #show the current frame and the fg masks
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    ## [show]

    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break